In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("../../data/data.json", "r", encoding="utf8"))


In [3]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [4]:
joined_text = df['title'] + df['text']
X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
Y = np.array(df['themes'].to_list())

In [5]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 1 simple Dense Layers

In [6]:
embedding_dim = 64

model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(500, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(100, activation='relu'),
  keras.layers.Dense(len(Y[0]), activation='sigmoid')
])

model1.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 2000)              19202000  
_________________________________________________________________
dropout (Dropout)            (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5

In [12]:
epochs = 5
checkpoint = keras.callbacks.ModelCheckpoint('val_classification_model.h5', 
    monitor='val_categorical_accuracy',
    mode='max',
    save_best_only=True)

model1.fit(np.array(X_train), np.array(Y_train),
          #batch_size=128,
          validation_data=(np.array(X_test),np.array(Y_test)),
          epochs=epochs,
          callbacks=[checkpoint])

Epoch 1/5
384/384 [==============================] - 84s 218ms/step - loss: 8.1100 - categorical_accuracy: 0.1770 - val_loss: 23.2832 - val_categorical_accuracy: 0.1832
Epoch 2/5
384/384 [==============================] - 87s 227ms/step - loss: 7.8486 - categorical_accuracy: 0.1768 - val_loss: 23.7998 - val_categorical_accuracy: 0.1832
Epoch 3/5
384/384 [==============================] - 90s 235ms/step - loss: 7.6444 - categorical_accuracy: 0.1794 - val_loss: 24.9804 - val_categorical_accuracy: 0.1832
Epoch 4/5
384/384 [==============================] - 92s 239ms/step - loss: 7.5349 - categorical_accuracy: 0.1797 - val_loss: 26.4753 - val_categorical_accuracy: 0.1832
Epoch 5/5
384/384 [==============================] - 89s 233ms/step - loss: 7.4472 - categorical_accuracy: 0.1831 - val_loss: 26.5339 - val_categorical_accuracy: 0.1832


In [10]:
score = model1.evaluate(np.array(X_test), np.array(Y_test)) 

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

43/43 [==============================] - 1s 14ms/step - loss: 21.4251 - categorical_accuracy: 0.1832
Test Score: 21.42511749267578
Test Accuracy: 0.18315018713474274


In [11]:
model1.save('classification_model.h5')

### Model 2 LSTM

In [10]:
embedding_dim =100 

model2 = keras.Sequential([
    layers.Embedding(max_features, embedding_dim, input_length=max_len),
    layers.SpatialDropout1D(0.2),
    layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(len(Y[0]), activation='sigmoid')
])

In [11]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [12]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 150, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 99)                9999      
Total params: 1,090,399
Trainable params: 1,090,399
Non-trainable params: 0
_________________________________________________________________


In [13]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string
batch_size = 128
epochs = 5

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
history = model2.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[early_stopping_callback])

Epoch 1/5
87/87 [==============================] - 17s 195ms/step - loss: 19.0051 - categorical_accuracy: 0.0032 - val_loss: 18.8436 - val_categorical_accuracy: 0.0032
Epoch 2/5
87/87 [==============================] - 17s 193ms/step - loss: 18.4259 - categorical_accuracy: 0.0017 - val_loss: 18.8219 - val_categorical_accuracy: 0.0032
Epoch 3/5
87/87 [==============================] - 17s 192ms/step - loss: 18.4121 - categorical_accuracy: 0.0017 - val_loss: 18.8022 - val_categorical_accuracy: 0.0032
Epoch 4/5
87/87 [==============================] - 17s 191ms/step - loss: 18.3958 - categorical_accuracy: 0.0017 - val_loss: 18.7963 - val_categorical_accuracy: 0.0032
Epoch 5/5
87/87 [==============================] - 17s 190ms/step - loss: 18.3771 - categorical_accuracy: 0.0017 - val_loss: 18.8038 - val_categorical_accuracy: 0.0032


In [14]:
accr = model2.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

43/43 [==============================] - 1s 15ms/step - loss: 18.5577 - categorical_accuracy: 0.0022
Test set
  Loss: 18.558
  Accuracy: 0.002


In [15]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(len(Y[0]), activation='softmax', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         89728     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [17]:
batch_size = 64
epochs = 5
model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/5
174/174 [==============================] - 5s 31ms/step - loss: 601521.3750 - accuracy: 0.0203 - val_loss: 3190682.5000 - val_accuracy: 0.0032
Epoch 2/5
174/174 [==============================] - 5s 29ms/step - loss: 55591072.0000 - accuracy: 0.0226 - val_loss: 164269184.0000 - val_accuracy: 0.0032
Epoch 3/5
174/174 [==============================] - 5s 29ms/step - loss: 633631616.0000 - accuracy: 0.0211 - val_loss: 1337953024.0000 - val_accuracy: 0.0032
Epoch 4/5
174/174 [==============================] - 5s 29ms/step - loss: 3085816064.0000 - accuracy: 0.0232 - val_loss: 5515577856.0000 - val_accuracy: 0.0170
Epoch 5/5
174/174 [==============================] - 5s 29ms/step - loss: 9826993152.0000 - accuracy: 0.0227 - val_loss: 15176954880.0000 - val_accuracy: 0.0032


In [18]:
accr = model3.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

43/43 [==============================] - 0s 4ms/step - loss: 15214229504.0000 - accuracy: 0.0022
Test set
  Loss: 15214229504.000
  Accuracy: 0.002


In [5]:
# one hot encoding for authors
def encode_authors(author_code):
    qty = df.author.max()
    result = [0] * (qty + 1)
    result[author_code] = 1
    return result

df.author = df.author.apply(encode_authors)

In [6]:
# one hot encoding for years
year_codes ={}
year_count = df.year.nunique()
def encode_year(year):
    result = [0] * year_count
    if year in year_codes.keys():
        result[year_codes[year]] = 1
    else:
        result[len(year_codes)] = 1
        year_codes[year] = len(year_codes)
    return result

df['years_encoded'] = df.year.apply(encode_year)

In [8]:
df['joined_text'] = df['text'] + df['title']
df['X2'] = df['author'] + df['years_encoded']
train_df, test_df = model_selection.train_test_split(df, test_size=0.1, random_state=42)
X1_train = keras.preprocessing.sequence.pad_sequences(train_df['joined_text'].to_list(), maxlen=max_len, padding='post')
X2_train = np.stack(train_df['X2'])
Y2_train = np.stack(train_df['themes'])
X1_test = keras.preprocessing.sequence.pad_sequences(list(test_df['joined_text']), maxlen=max_len, padding='post')
X2_test = np.stack(test_df['X2'])
Y2_test = np.stack(test_df['themes'])

In [11]:
text_input = keras.Input(shape=(max_len,))
categorical_input = keras.Input(shape=(len(X2_train[0]),))

text_embedding = layers.Embedding(max_features, 64)(text_input)
categorical_embedding = layers.Embedding(2, 8)(categorical_input)

flat_text = layers.Flatten()(text_embedding)
flat_categories = layers.Flatten()(categorical_embedding)

concatenated = keras.layers.Concatenate()([flat_text, flat_categories])

dense1 = keras.layers.Dense(2000, activation='relu', )(concatenated)
dense2 = keras.layers.Dense(500, activation='relu', )(dense1)
dense3 = keras.layers.Dense(100, activation='relu', )(dense2)
out = keras.layers.Dense(len(Y2_train[0]), activation='sigmoid')(dense3)


united_model = keras.Model(inputs=[text_input, categorical_input], outputs=out)


In [12]:
united_model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
united_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4299)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 150, 64)      640000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 4299, 8)      16          input_2[0][0]                    
______________________________________________________________________________________________

In [14]:
united_model.fit([X1_train, X2_train], Y2_train, epochs=3, validation_split=0.1)

Epoch 1/3
347/347 [==============================] - 317s 913ms/step - loss: 15.5480 - categorical_accuracy: 0.0938 - val_loss: 18.4029 - val_categorical_accuracy: 0.0812
Epoch 2/3
347/347 [==============================] - 316s 909ms/step - loss: 12.9274 - categorical_accuracy: 0.2240 - val_loss: 19.8288 - val_categorical_accuracy: 0.0779
Epoch 3/3
347/347 [==============================] - 315s 908ms/step - loss: 11.1095 - categorical_accuracy: 0.3113 - val_loss: 21.3777 - val_categorical_accuracy: 0.0828


In [15]:
score3 = united_model.evaluate([np.array(X1_test), np.array(X2_test)], np.array(Y2_test)) 

print("Test Score:", score3[0])
print("Test Accuracy:", score3[1])

43/43 [==============================] - 3s 61ms/step - loss: 21.2575 - categorical_accuracy: 0.0840
Test Score: 21.25745391845703
Test Accuracy: 0.08400291949510574


In [23]:
df

,title,author,year,text,themes,joined_text,X2,years_encoded
0,"[45, 4, 141, 903]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2002,"[2, 2334, 1097, 5, 2334, 298, 9780, 1002, 412,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2334, 1097, 5, 2334, 298, 9780, 1002, 412,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[2828],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2002,"[1, 95, 106, 4385, 3, 144, 291, 33, 1426, 8803...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 95, 106, 4385, 3, 144, 291, 33, 1426, 8803...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[209],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2002,"[106, 20, 719, 21, 1474, 329, 240, 159, 421, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[106, 20, 719, 21, 1474, 329, 240, 159, 421, 2...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1210, 40, 114, 1989, 18, 2321]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2002,"[329, 303, 17, 70, 8, 1, 8, 1, 43, 1345, 1, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[329, 303, 17, 70, 8, 1, 8, 1, 43, 1345, 1, 20...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[259, 1210]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2002,"[9, 649, 76, 140, 544, 206, 679, 179, 107, 44,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9, 649, 76, 140, 544, 206, 679, 179, 107, 44,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
17067,"[151, 18, 2, 5178]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1990,"[1, 395, 2499, 331, 545, 8944, 28, 2573, 93, 8...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[1, 395, 2499, 331, 545, 8944, 28, 2573, 93, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17068,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1990,"[2, 597, 152, 153, 200, 277, 167, 4114, 3080, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[2, 597, 152, 153, 200, 277, 167, 4114, 3080, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17070,"[15, 114]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1990,"[114, 17, 70, 125, 628, 2, 797, 1, 99, 1133, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[114, 17, 70, 125, 628, 2, 797, 1, 99, 1133, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17071,"[15, 25, 1, 13]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2005,"[15, 1, 13, 37, 36, 15, 1, 311, 174, 544, 38, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[15, 1, 13, 37, 36, 15, 1, 311, 174, 544, 38, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
